In [82]:
import catboost
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, sum_models

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [44]:
df = pd.read_csv("indicators_buy.csv")
df = df.drop(df.columns[0], axis=1).fillna(-1)
df

,date,open,high,low,close,volume,buy,sell,pair,adx,ao,macd,macdsignal,macdhist,strat_buy_signal_AwesomeMacd,lower,bbdelta,closedelta,tail,bb_lowerband,bb_middleband,ema_slow,volume_mean_slow,strat_buy_signal_CombinedBinHAndCluc,mid,bb_upperband,strat_buy_signal_CombinedBinHAndClucV5,date_x,open_x,high_x,low_x,close_x,volume_x,ema_50_x,ema_200_x,rsi_x,ssl-up,ssl-down,ssl-dir,ema_50,ema_200,sma_5,rsi,strat_buy_signal_CombinedBinHAndClucV6H,date_y,open_y,high_y,low_y,close_y,volume_y,ema_50_y,ema_200_y,sma_200_x,rsi_y,ssl_down_x,ssl_up_x,sma_200,mfi,strat_buy_signal_CombinedBinHAndClucV7,date_x.1,open_x.1,high_x.1,low_x.1,close_x.1,volume_x.1,ema_50.1,ema_100,ema_200.1,sma_200_y,sma_200_dec,rsi.1,ssl_down_y,ssl_up_y,ema_12,ema_26,sma_200_dec.1,strat_buy_signal_CombinedBinHAndClucV8,ma_offset_buy,ma_offset_sell,strat_buy_signal_SMAOffset,date_y.1,open_y.1,high_y.1,low_y.1,close_y.1,volume_y.1,ema_fast,ema_slow.1,go_long,sma_30_offset,sma_30_offset_pos,strat_buy_signal_SMAOffsetV2,date_1h_x,open_1h_x,high_1h_x,low_1h_x,close_1h_x,volume_1h_x,ema_20_1h_x,ema_50_1h_x,ema_200_1h_x,rsi_1h_x,ssl_down_1h_x,ssl_up_1h_x,sma_9,strat_buy_signal_NostalgiaForInfinityV1,date_1h_y,open_1h_y,high_1h_y,low_1h_y,close_1h_y,volume_1h_y,ema_20_1h_y,ema_50_1h_y,ema_200_1h_y,rsi_1h_y,ssl_down_1h_y,ssl_up_1h_y,strat_buy_signal_NostalgiaForInfinityV2,date_1h,open_1h,high_1h,low_1h,close_1h,volume_1h,ema_15_1h,ema_50_1h,ema_100_1h,ema_200_1h,sma_50_1h,sma_200_1h,rsi_1h,bb_lowerband_1h,bb_middleband_1h,bb_upperband_1h,volume_mean_30,lips,smma_lips,teeth,smma_teeth,jaw,smma_jaw,volume_mean_4,safe_dips,strat_buy_signal_NostalgiaForInfinityV3
0,2021-05-21 00:00:00+00:00,466.85,476.22,464.94,475.53,1547.5270,0,0,AAVE/USDT,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0,-1.000000,-1.000000,-1.00,10.59,-1.000000,472.230000,-1.000000,-1.000000,0,475.530000,-1.000000,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1,-1.000000,-1.000000,-1.000,-1.000000,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.00,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.00,False,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,False,0,-1.000000,-1.000000,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.0,-1.00000,-1.000000,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.00,-1.000000,-1.000000,-1.000,-1.000000,-1.000000,-1.000,-1.000000,-1.00000,-1.00000,-1.000000,-1.000000,-1.000000,False,0
1,2021-05-21 00:05:00+00:00,476.12,476.22,471.67,474.37,1238.4810,0,0,AAVE/USDT,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0,473.309512,1.640488,1.16,2.70,470.532610,473.158333,-1.000000,-1.000000,0,474.950000,475.784057,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1,-1.000000,-1.000000,-1.000,-1.000000,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.00,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.00,False,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,False,0,-1.000000,-1.000000,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.0,-1.00000,-1.000000,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0,-1,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.00,-1.000000,-1.000000,-1.000,-1.000000,-1.000000,-1.000,-1.000000,-1.00000,-1.00000,-1.000000,-1.000000,-1.000000,False,0
2,2021-05-21 00:10:00+00:00,474.26,478.29,473.63,476.53,910.1689,0,0,AAVE/USDT,-1.000000,-1.000000,-1.000000,-1.000000,-1.

In [66]:
df["strat_buy_signal"] = np.random.randint(0, 2, size=len(df))
y = df["strat_buy_signal"]
x = df[[col for col in df.columns if "strat_buy_signal" not in col]]
x = x[[col for col in x.columns if "date" not in col]]

In [125]:
cat_features=[i for i in x.columns if x.dtypes[i]=='object']
display(cat_features)
display(x)
display(y)

['pair', 'ssl-dir']

,open,high,low,close,volume,buy,sell,pair,adx,ao,macd,macdsignal,macdhist,lower,bbdelta,closedelta,tail,bb_lowerband,bb_middleband,ema_slow,volume_mean_slow,mid,bb_upperband,open_x,high_x,low_x,close_x,volume_x,ema_50_x,ema_200_x,rsi_x,ssl-up,ssl-down,ssl-dir,ema_50,ema_200,sma_5,rsi,open_y,high_y,low_y,close_y,volume_y,ema_50_y,ema_200_y,sma_200_x,rsi_y,ssl_down_x,ssl_up_x,sma_200,mfi,open_x.1,high_x.1,low_x.1,close_x.1,volume_x.1,ema_50.1,ema_100,ema_200.1,sma_200_y,sma_200_dec,rsi.1,ssl_down_y,ssl_up_y,ema_12,ema_26,sma_200_dec.1,ma_offset_buy,ma_offset_sell,open_y.1,high_y.1,low_y.1,close_y.1,volume_y.1,ema_fast,ema_slow.1,go_long,sma_30_offset,sma_30_offset_pos,open_1h_x,high_1h_x,low_1h_x,close_1h_x,volume_1h_x,ema_20_1h_x,ema_50_1h_x,ema_200_1h_x,rsi_1h_x,ssl_down_1h_x,ssl_up_1h_x,sma_9,open_1h_y,high_1h_y,low_1h_y,close_1h_y,volume_1h_y,ema_20_1h_y,ema_50_1h_y,ema_200_1h_y,rsi_1h_y,ssl_down_1h_y,ssl_up_1h_y,open_1h,high_1h,low_1h,close_1h,volume_1h,ema_15_1h,ema_50_1h,ema_100_1h,ema_200_1h,sma_50_1h,sma_200_1h,rsi_1h,bb_lowerband_1h,bb_middleband_1h,bb_upperband_1h,volume_mean_30,lips,smma_lips,teeth,smma_teeth,jaw,smma_jaw,volume_mean_4,safe_dips
0,466.85,476.22,464.94,475.53,1547.5270,0,0,AAVE/USDT,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00,10.59,-1.000000,472.230000,-1.000000,-1.000000,475.530000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1,-1.000000,-1.000000,-1.000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.00,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.00,False,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,False,-1.000000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.0,-1.00000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.00,-1.000000,-1.000000,-1.000,-1.000000,-1.000000,-1.000,-1.000000,-1.00000,-1.00000,-1.000000,-1.000000,-1.000000,False
1,476.12,476.22,471.67,474.37,1238.4810,0,0,AAVE/USDT,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,473.309512,1.640488,1.16,2.70,470.532610,473.158333,-1.000000,-1.000000,474.950000,475.784057,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1,-1.000000,-1.000000,-1.000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.00,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.00,False,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,False,-1.000000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.0,-1.00000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.00,-1.000000,-1.000000,-1.000,-1.000000,-1.000000,-1.000,-1.000000,-1.00000,-1.00000,-1.000000,-1.000000,-1.000000,False
2,474.26,478.29,473.63,476.53,910.1689,0,0,AAVE/USDT,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,473.314692,2.161974,2.16,2.90,470.233740,474.155556,-1.000000,-1.000000,475.476667,478.077371,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1,-1.000000,-1.000000,-1.000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.00,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.00,False,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,False,-1.000000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.0,-1.00000,-1.000000,-1.00,-1.00,-1.00,-1.00,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,

0       1
1       1
2       0
3       0
4       0
       ..
4865    0
4866    1
4867    1
4868    0
4869    0
Name: strat_buy_signal, Length: 4870, dtype: int64

In [72]:
dataset = Pool(data=x, label=y, cat_features=cat_features)

In [73]:
# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=100)
# Fit model with `use_best_model=True`

model.fit(dataset,
          use_best_model=True,
          eval_set=dataset)

print("Count of trees in model = {}".format(model.tree_count_))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.127564
0:	learn: 0.6919558	test: 0.6919558	best: 0.6919558 (0)	total: 114ms	remaining: 11.3s
1:	learn: 0.6911623	test: 0.6911623	best: 0.6911623 (1)	total: 123ms	remaining: 6.03s
2:	learn: 0.6905800	test: 0.6905800	best: 0.6905800 (2)	total: 133ms	remaining: 4.29s
3:	learn: 0.6884488	test: 0.6884488	best: 0.6884488 (3)	total: 142ms	remaining: 3.42s
4:	learn: 0.6873105	test: 0.6873105	best: 0.6873105 (4)	total: 153ms	remaining: 2.91s
5:	learn: 0.6856059	test: 0.6856059	best: 0.6856059 (5)	total: 164ms	remaining: 2.56s
6:	learn: 0.6844173	test: 0.6844173	best: 0.6844173 (6)	total: 173ms	remaining: 2.3s
7:	learn: 0.6836586	test: 0.6836586	best: 0.6836586 (7)	total: 185ms	remaining: 2.13s
8:	learn: 0.6827147	test: 0.6827147	best: 0.6827147 (8)	total: 198ms	remaining: 2s
9:	learn: 0.6814760	test: 0.6814760	best: 0.6814760 (9)	total: 208ms	remaining: 1.88s
10:	learn: 0.6805304	test: 0.6805304	best: 0.6805304 (10)	total: 219ms	remaining: 1.77s
11:	learn: 0.6799203	test:

In [81]:
print("classes", model.classes_)
model.predict_proba(dataset)

classes [0 1]


array([[0.16292992, 0.83707008],
       [0.43741867, 0.56258133],
       [0.65072511, 0.34927489],
       ...,
       [0.45570678, 0.54429322],
       [0.56471163, 0.43528837],
       [0.4870747 , 0.5129253 ]])

In [83]:
dataset2 = Pool(data=x, label=y, cat_features=cat_features)
model2 = CatBoostClassifier(iterations=100)
model2.fit(dataset,
          use_best_model=True,
          eval_set=dataset)

print("Count of trees in model = {}".format(model.tree_count_))

Learning rate set to 0.127564
0:	learn: 0.6919558	test: 0.6919558	best: 0.6919558 (0)	total: 17.1ms	remaining: 1.69s
1:	learn: 0.6911623	test: 0.6911623	best: 0.6911623 (1)	total: 30.9ms	remaining: 1.52s
2:	learn: 0.6905800	test: 0.6905800	best: 0.6905800 (2)	total: 45.6ms	remaining: 1.47s
3:	learn: 0.6884488	test: 0.6884488	best: 0.6884488 (3)	total: 59.3ms	remaining: 1.42s
4:	learn: 0.6873105	test: 0.6873105	best: 0.6873105 (4)	total: 71.7ms	remaining: 1.36s
5:	learn: 0.6856059	test: 0.6856059	best: 0.6856059 (5)	total: 84.4ms	remaining: 1.32s
6:	learn: 0.6844173	test: 0.6844173	best: 0.6844173 (6)	total: 95.5ms	remaining: 1.27s
7:	learn: 0.6836586	test: 0.6836586	best: 0.6836586 (7)	total: 107ms	remaining: 1.23s
8:	learn: 0.6827147	test: 0.6827147	best: 0.6827147 (8)	total: 118ms	remaining: 1.2s
9:	learn: 0.6814760	test: 0.6814760	best: 0.6814760 (9)	total: 129ms	remaining: 1.16s
10:	learn: 0.6805304	test: 0.6805304	best: 0.6805304 (10)	total: 143ms	remaining: 1.16s
11:	learn: 0.679

In [100]:
models = [model, model2]
final_model = sum_models(models, weights=None, ctr_merge_policy='IntersectingCountersAverage')
print("classes", model.classes_)

classes [0 1]


In [109]:
preds = final_model.predict(dataset, prediction_type="Probability")
preds

array([[0.03650299, 0.96349701],
       [0.37676786, 0.62323214],
       [0.77633871, 0.22366129],
       ...,
       [0.41210334, 0.58789666],
       [0.62729109, 0.37270891],
       [0.47416667, 0.52583333]])

In [124]:
df["buy_proba"] = preds[:,1]
df["buy"] = df['buy_proba'].apply(lambda x: 1 if x > 0.5 else 0)
df[["buy", "buy_proba"]]

,buy,buy_proba
0,1,0.963497
1,1,0.623232
2,0,0.223661
3,0,0.463407
4,0,0.119105
...,...,...
4865,0,0.228030
4866,1,0.597624
4867,1,0.587897
4868,0,0.372709


In [116]:
final_model.save_model("test_model")

In [120]:
loaded_model = CatBoostClassifier(iterations=100)
loaded_model = loaded_model.load_model("test_modelss")
preds = final_model.predict(dataset, prediction_type="Probability")
preds

CatBoostError: catboost/libs/model/model_import_interface.h:19: Model file doesn't exist: test_modelss